In [ ]:
# connect to drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
cd gdrive/MyDrive/SemTech4STLD_related_code_and_data/

/content/gdrive/MyDrive/SemTech4STLD_related_code_and_data


In [ ]:
from google.colab import files
files.upload()

In [ ]:
ls

'Copy of ClaimCiteRetrieval_ESR_based_JSONed_20-02-2024_new.xlsx'
'Copy of ClaimCiteRetrieval_ESR_based_JSONed_20-02-2024.xlsx'
'Copy of ClaimCiteRetrieval_ESR_based_JSONed_24-02-2024.xlsx'
 sample-dataset-for-results-analysis-SemTech4stld.xlsx


# Novelty passage retrieval

In [ ]:
# Read xl file
import pandas as pd
csv_file_path = 'Copy of ClaimCiteRetrieval_ESR_based_JSONed_24-02-2024.xlsx'
df = pd.read_excel(csv_file_path)

In [ ]:
df.head(2)

,APnum,PAnum,APclaim,PAclaim,APabstract,PAabstract,APipc,PAipc,PAparagraphs,PApragraphs_Json,...,PApredGPT_top2,PApredGPT_top3,PApredBARD_all3,PApredBARD_top1,PApredBARD_top2,PApredBARD_top3,APipc_GPTpred,APipc_BARDpred,APipc_abstract_GPTpred,APipc_abstract_BARDpred
0,EP4300511,US20140019162A1,Text classification method to aid on clinical ...,"An automated triage system, comprising:\na pat...",The present application describes an automatic...,An automated triage system may include a patie...,"G16H50/20, G16H10/60, G16H40/20, G06F40/10, G0...","G06F 9/00, G06O 50/24,",[0057]\nFIGS. 3A-3B illustrate a more detailed...,"{\n ""sentences"": [\n {\n ""id"": ""0057""...",...,NaN,NaN,"\n(0058, 0059, 0092)",NaN,NaN,NaN,"G16H 30/20, G06Q 50/24, A61B 5/00, G06F 19/00,...","ipc1: G06F19/00, ipc2: G06N3/08, ipc3: G16H50/...","G06F17/305, G16H30/20, A61B5/1503, G06N3/00, G...","G06F17/30, G06F7/07, G16H50/20, G16H50/50, G06..."
1,EP4273738,US20080091405A1,A semantic representation generation method of...,A method of analyzing a sentence in a source l...,In a semantic representation generation method...,A method and computer system for analyzing sen...,"G06F40/30, // G06F40/211, G06F40/268, ...",G06F 7/28,\n[0004]\nThe acquired ability to understand...,"{\n ""sentences"": [\n {\n ""id"": ""0004""...",...,NaN,NaN,"(0004, 0005, 0015)",NaN,NaN,NaN,"G06F 17/27, G06F 17/28, G06F 17/30, G06F 17/50...","\nipc1: G06N3/05, ipc2: G10L15/00, ipc3: H04L2...","G06F17/27, G06N3/04, G06F17/305, G06Q10/08, G0...","G10L15/00, G10L15/02, G10L25/00, G10L25/04, G0..."


### Task 1: Cleaning IPC values

In [ ]:

# Extract the string values without converting them to integers
df['PAcited'] = df['PAcited'].str.extractall(r'(\d+)').astype(str).groupby(level=0).agg(','.join)

# Convert the comma-separated string to a list of strings
df['PAcited'] = df['PAcited'].apply(lambda x: [f'{num:04d}' for num in map(int, x.split(','))])


# Extract the string values without converting them to integers
df['PApredGPT_all3'] = df['PApredGPT_all3'].str.extractall(r'(\d+)').astype(str).groupby(level=0).agg(','.join)

# Convert the comma-separated string to a list of strings with leading zeros preserved
df['PApredGPT_all3'] = df['PApredGPT_all3'].apply(lambda x: [f'{num:04d}' for num in map(int, x.split(','))])


# Extract the string values without converting them to integers
df['PApredBARD_all3'] = df['PApredBARD_all3'].str.extractall(r'(\d+)').astype(str).groupby(level=0).agg(','.join)

# Convert the comma-separated string to a list of strings with leading zeros preserved
df['PApredBARD_all3'] = df['PApredBARD_all3'].apply(lambda x: [f'{num:04d}' for num in map(int, x.split(','))])




*Verification after converting to single list*

In [ ]:
df['PAcited'][0]

['0060', '0061', '0092']

In [ ]:
df['PApredGPT_all3'][0]

['0058', '0059', '0061']

In [ ]:
df['PApredBARD_all3'][0]

['0058', '0059', '0092']

# Calculate Top1, Top2, Top3 accuracies of passage retrieval

### Top1 accuracies

In [ ]:
# Column to compare
predicted_column = 'PApredGPT_all3'
truth_column = 'PAcited'

# Function to calculate accuracy
def calculate_accuracy(row):
    predicted_list = row[predicted_column][0]
    truth_list = row[truth_column]

    # Check if any item in the truth list matches the first item in the predicted list
    return any(truth_item == predicted_list for truth_item in truth_list)

# Apply the accuracy calculation function to the predicted column and calculate the mean accuracy
accuracy_col = f'Top1_accuracy_of_{predicted_column}'
accuracy = df.apply(calculate_accuracy, axis=1).mean()

# Print the accuracy for the predicted column
print(f'{accuracy_col}: {accuracy:.2%}')


Top1_accuracy_of_PApredGPT_all3: 55.10%


In [ ]:
# Column to compare
predicted_column = 'PApredBARD_all3'
truth_column = 'PAcited'

# Function to calculate accuracy
def calculate_accuracy(row):
    predicted_list = row[predicted_column][0]
    truth_list = row[truth_column]

    # Check if any item in the truth list matches the first item in the predicted list
    return any(truth_item == predicted_list for truth_item in truth_list)

# Apply the accuracy calculation function to the predicted column and calculate the mean accuracy
accuracy_col = f'Top1_accuracy_of_{predicted_column}'
accuracy = df.apply(calculate_accuracy, axis=1).mean()

# Print the accuracy for the predicted column
print(f'{accuracy_col}: {accuracy:.2%}')

Top1_accuracy_of_PApredBARD_all3: 48.98%


### Top2 accuracies

In [ ]:
# Column to compare
predicted_column = 'PApredGPT_all3'
truth_column = 'PAcited'

# Function to calculate accuracy
def calculate_accuracy(row):
    predicted_list = row[predicted_column][1]
    truth_list = row[truth_column]

    # Check if any item in the truth list matches the first item in the predicted list
    return any(truth_item == predicted_list for truth_item in truth_list)

# Apply the accuracy calculation function to the predicted column and calculate the mean accuracy
accuracy_col = f'Top2_accuracy_of_{predicted_column}'
accuracy = df.apply(calculate_accuracy, axis=1).mean()

# Print the accuracy for the predicted column
print(f'{accuracy_col}: {accuracy:.2%}')

Top2_accuracy_of_PApredGPT_all3: 48.98%


In [ ]:
  # Column to compare
predicted_column = 'PApredBARD_all3'
truth_column = 'PAcited'

# Function to calculate accuracy
def calculate_accuracy(row):
    predicted_list = row[predicted_column][1]
    truth_list = row[truth_column]

    # Check if any item in the truth list matches the first item in the predicted list
    return any(truth_item == predicted_list for truth_item in truth_list)

# Apply the accuracy calculation function to the predicted column and calculate the mean accuracy
accuracy_col = f'Top2_accuracy_of_{predicted_column}'
accuracy = df.apply(calculate_accuracy, axis=1).mean()

# Print the accuracy for the predicted column
print(f'{accuracy_col}: {accuracy:.2%}')

Top2_accuracy_of_PApredBARD_all3: 40.82%


### Top3 accuracies

In [ ]:
# Column to compare
predicted_column = 'PApredGPT_all3'
truth_column = 'PAcited'

# Function to calculate accuracy
def calculate_accuracy(row):
    predicted_list = row[predicted_column][2]
    truth_list = row[truth_column]

    # Check if any item in the truth list matches the first item in the predicted list
    return any(truth_item == predicted_list for truth_item in truth_list)

# Apply the accuracy calculation function to the predicted column and calculate the mean accuracy
accuracy_col = f'Top3_accuracy_of_{predicted_column}'
accuracy = df.apply(calculate_accuracy, axis=1).mean()

# Print the accuracy for the predicted column
print(f'{accuracy_col}: {accuracy:.2%}')

Top3_accuracy_of_PApredGPT_all3: 44.90%


In [ ]:
  # Column to compare
predicted_column = 'PApredBARD_all3'
truth_column = 'PAcited'

# Function to calculate accuracy
def calculate_accuracy(row):
    predicted_list = row[predicted_column][2]
    truth_list = row[truth_column]

    # Check if any item in the truth list matches the first item in the predicted list
    return any(truth_item == predicted_list for truth_item in truth_list)

# Apply the accuracy calculation function to the predicted column and calculate the mean accuracy
accuracy_col = f'Top3_accuracy_of_{predicted_column}'
accuracy = df.apply(calculate_accuracy, axis=1).mean()

# Print the accuracy for the predicted column
print(f'{accuracy_col}: {accuracy:.2%}')

Top3_accuracy_of_PApredBARD_all3: 51.02%


In [ ]:

df.head(2)

,APnum,PAnum,APclaim,PAclaim,APabstract,PAabstract,APipc,PAipc,PAparagraphs,PApragraphs_Json,...,PApredGPT_top2,PApredGPT_top3,PApredBARD_all3,PApredBARD_top1,PApredBARD_top2,PApredBARD_top3,APipc_GPTpred,APipc_BARDpred,APipc_abstract_GPTpred,APipc_abstract_BARDpred
0,EP4300511,US20140019162A1,Text classification method to aid on clinical ...,"An automated triage system, comprising:\na pat...",The present application describes an automatic...,An automated triage system may include a patie...,"G16H50/20, G16H10/60, G16H40/20, G06F40/10, G0...","G06F 9/00, G06O 50/24,",[0057]\nFIGS. 3A-3B illustrate a more detailed...,"{\n ""sentences"": [\n {\n ""id"": ""0057""...",...,NaN,NaN,"[0058, 0059, 0092]",NaN,NaN,NaN,"G16H 30/20, G06Q 50/24, A61B 5/00, G06F 19/00,...","ipc1: G06F19/00, ipc2: G06N3/08, ipc3: G16H50/...","G06F17/305, G16H30/20, A61B5/1503, G06N3/00, G...","G06F17/30, G06F7/07, G16H50/20, G16H50/50, G06..."
1,EP4273738,US20080091405A1,A semantic representation generation method of...,A method of analyzing a sentence in a source l...,In a semantic representation generation method...,A method and computer system for analyzing sen...,"G06F40/30, // G06F40/211, G06F40/268, ...",G06F 7/28,\n[0004]\nThe acquired ability to understand...,"{\n ""sentences"": [\n {\n ""id"": ""0004""...",...,NaN,NaN,"[0004, 0005, 0015]",NaN,NaN,NaN,"G06F 17/27, G06F 17/28, G06F 17/30, G06F 17/50...","\nipc1: G06N3/05, ipc2: G10L15/00, ipc3: H04L2...","G06F17/27, G06N3/04, G06F17/305, G06Q10/08, G0...","G10L15/00, G10L15/02, G10L25/00, G10L25/04, G0..."


# Creating PAtotal column to find all candidate paragraphs

In [ ]:
import pandas as pd
import json

# Assuming your DataFrame is named 'df'
# If not, replace 'df' with your DataFrame's actual name

# Function to extract 'id' values from JSON string
def extract_ids(json_str):
    try:
        data = json.loads(json_str)
        return [sentence["id"] for sentence in data["sentences"]]
    except (json.JSONDecodeError, KeyError):
        return []

# Apply the function to create a new column 'PAtotal'
df['PAtotal'] = df['PApragraphs_Json'].apply(extract_ids)



In [ ]:
df['PAtotal']

0     [0057, 0058, 0059, 0060, 0061, 0092, 0093, 009...
1     [0004, 0005, 0007, 0008, 0009, 0010, 0015, 001...
2     [0001, 0002, 0003, 0004, 0005, 0006, 0039, 004...
3      [0003, 0004, 0005, 0007, 0008, 0009, 0017, 0019]
4     [0022, 0023, 0024, 0025, 0026, 0027, 0037, 003...
5     [0225, 0226, 0227, 0228, 0224, 0287, 0288, 028...
6     [0006, 0007, 0008, 0009, 0026, 0027, 0028, 002...
7     [0008, 0009, 0010, 0011, 0021, 0022, 0023, 004...
8     [0027, 0028, 0029, 0030, 0031, 0034, 0035, 003...
9     [0001, 0002, 0003, 0004, 0005, 0006, 0007, 000...
10    [0009, 0010, 0011, 0012, 0034, 0035, 0036, 003...
11     [0003, 0016, 0017, 0018, 0022, 0023, 0024, 0025]
12    [0003, 0004, 0005, 0006, 0014, 0015, 0016, 001...
13    [0005, 0006, 0007, 0018, 0023, 0024, 0025, 003...
14    [0039, 0040, 0041, 0042, 0043, 0046, 0047, 004...
15    [0001, 0045, 0002, 0003, 0004, 0005, 0006, 000...
16    [0142, 0143, 0144, 0168, 0169, 0172, 0182, 018...
17    [0008, 0009, 0010, 0011, 0012, 0013, 0026,

In [ ]:
# Assuming 'df' is your DataFrame with the 'PAtotal' column

# Iterate through 'PAtotal' column and identify rows with less than 9 items
for index, ids_list in enumerate(df['PAtotal']):
    if len(ids_list) < 9:
        print(f"Row {index} has less than 9 items: {ids_list}")


Row 3 has less than 9 items: ['0003', '0004', '0005', '0007', '0008', '0009', '0017', '0019']
Row 11 has less than 9 items: ['0003', '0016', '0017', '0018', '0022', '0023', '0024', '0025']
Row 19 has less than 9 items: ['0014', '0015', '0016', '0055', '0056', '0057']
Row 29 has less than 9 items: ['0001', '0002', '0014', '0018', '0013', '0004', '0005', '0006']


# Task2: Calcuate precison, recall, F1 scores

In [ ]:

import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate precision, recall, F1 score, and accuracy
def calculate_performance(row):
    # Extract actual cited values, predicted values, and total values from the row
    actual_cited = row['PAcited']
    predicted_values = row['PApredGPT_all3']
    total_values = row['PAtotal']

    # Calculate metrics
    precision = precision_score(actual_cited, predicted_values, average='micro')
    recall = recall_score(actual_cited, predicted_values, average='micro')
    f1 = f1_score(actual_cited, predicted_values, average='micro')
    accuracy = accuracy_score(actual_cited, predicted_values)

    # Calculate additional metrics based on total values
    true_positives = len(set(actual_cited).intersection(set(predicted_values)))
    false_positives = len(set(predicted_values).difference(set(actual_cited)))
    false_negatives = len(set(actual_cited).difference(set(predicted_values)))
    true_negatives = len(set(total_values).difference(set(actual_cited).union(set(predicted_values))))

    precision_macro = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    recall_macro = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
    f1_macro = 2 * (precision_macro * recall_macro) / (precision_macro + recall_macro) if (precision_macro + recall_macro) != 0 else 0

    return precision, recall, f1, accuracy, precision_macro, recall_macro, f1_macro

# Apply the performance calculation function to each row and create a new DataFrame with results
performance_df = df.apply(calculate_performance, axis=1, result_type='expand')
performance_df.columns = ['Precision', 'Recall', 'F1 Score', 'Accuracy', 'Precision Macro', 'Recall Macro', 'F1 Macro']

# Calculate mean values across all rows
overall_performance = performance_df.mean()


print("\nOverall Performance Metrics:")
print(overall_performance)


Overall Performance Metrics:
Precision          0.251701
Recall             0.251701
F1 Score           0.251701
Accuracy           0.251701
Precision Macro    0.496599
Recall Macro       0.496599
F1 Macro           0.496599
dtype: float64


In [ ]:

import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
# Function to calculate precision, recall, F1 score, and accuracy
def calculate_performance(row):
    # Extract actual cited values, predicted values, and total values from the row
    actual_cited = row['PAcited']
    predicted_values = row['PApredGPT_all3']
    total_values = row['PAtotal']

    # Calculate metrics
    precision = precision_score(actual_cited, predicted_values, average='micro')
    recall = recall_score(actual_cited, predicted_values, average='micro')
    f1 = f1_score(actual_cited, predicted_values, average='micro')
    accuracy = accuracy_score(actual_cited, predicted_values)

    # Calculate additional metrics based on total values
    true_positives = len(set(actual_cited).intersection(set(predicted_values)))
    false_positives = len(set(predicted_values).difference(set(actual_cited)))
    false_negatives = len(set(actual_cited).difference(set(predicted_values)))
    true_negatives = len(set(total_values).difference(set(actual_cited).union(set(predicted_values))))

    precision_macro = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    recall_macro = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
    f1_macro = 2 * (precision_macro * recall_macro) / (precision_macro + recall_macro) if (precision_macro + recall_macro) != 0 else 0

    return precision, recall, f1, accuracy, precision_macro, recall_macro, f1_macro

# Apply the performance calculation function to each row and create a new DataFrame with results
performance_df = df.apply(calculate_performance, axis=1, result_type='expand')
performance_df.columns = ['Precision', 'Recall', 'F1 Score', 'Accuracy', 'Precision Macro', 'Recall Macro', 'F1 Macro']

# Display the original DataFrame and the performance DataFrame
#print("Original DataFrame:")
#print(df)

print("\nPerformance DataFrame:")
print(performance_df)


Performance DataFrame:
    Precision    Recall  F1 Score  Accuracy  Precision Macro  Recall Macro  \
0    0.000000  0.000000  0.000000  0.000000         0.333333      0.333333   
1    0.333333  0.333333  0.333333  0.333333         0.666667      0.666667   
2    0.333333  0.333333  0.333333  0.333333         0.666667      0.666667   
3    0.333333  0.333333  0.333333  0.333333         0.333333      0.333333   
4    0.333333  0.333333  0.333333  0.333333         1.000000      1.000000   
5    0.000000  0.000000  0.000000  0.000000         0.333333      0.333333   
6    0.000000  0.000000  0.000000  0.000000         0.333333      0.333333   
7    0.333333  0.333333  0.333333  0.333333         0.333333      0.333333   
8    0.333333  0.333333  0.333333  0.333333         0.333333      0.333333   
9    0.333333  0.333333  0.333333  0.333333         0.333333      0.333333   
10   0.000000  0.000000  0.000000  0.000000         0.333333      0.333333   
11   0.333333  0.333333  0.333333  0.333